# Importation des données
[To Do : continuer de décrire origine et format des données]

Les données sont localisées dans des fichiers XML, chaque fichier correspondant à une demande de brevets. Ces fichiers sont directement téléchargeables depuis le serveur FTP de l'INPI, accessible via FileZilla.

Pour rendre notre projet reproductible au maximum, bien que les données ne puissent pas être directement téléchargées par un utilisateur non autorisé par l'INPI, nous avons stocké les fichiers .zip tels que nous les avons initialement téléchargés dans le bucket diffusion de l'un de nous. 

Concernant leur format, ces fichiers compressés contiennent chacun les informations sur toutes les demandes de brevets déposés auprès de l'INPI chaque semaine. Il y a donc 52 dossiers par année, un pour chaque semaine. Dans ces dossiers, se trouvent des informations dont nous n'avons pas besoin (ex: des schémas d'invention). Nous n'utiliserons que les données présentes dans des fichiers XML, chacun de ces fichiers contenant de nombreuses informations sur une demande de brevet déposée dans la semaine concernée.

Le code suivant se charge de décompresser ces fichiers, d'identifier les fichiers XML au sein de leurs arborescences et d'en extraire toutes les informations.

In [1]:
import os
import s3fs
import pandas as pd

from scripts.importation import process_all_years_s3

In [2]:
# Connexion au bucket S3
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

ROOT_S3_PATH = "mvallat/diffusion/projet"

In [ ]:
# NB : ce code tourne +7min
data_brevets = process_all_years_s3(fs, ROOT_S3_PATH)
data_brevets.head(2)

/home/onyxia/work/Projet-Python/scripts/importation.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

    ⚠️ Erreur sur FR_FRNEWST36_2022_25/doc/3118340.xml: no element found: line 1, column 0


/home/onyxia/work/Projet-Python/scripts/importation.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_year["year"] = year_name
/home/onyxia/work/Projet-Python/scripts/importation.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

,family_id,doc-number,kind,country,date-produced,status,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,...,publication_doc-number,publication_date,publication_bopinum,publication_nature,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,NA,NA,NA,NA,20161223,NA,NA,NA,NA,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA,NA,NA,NA,20161230,NA,NA,NA,NA,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Exportation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_OUT_S3, 'wb') as file_out:
    data_brevets.to_parquet(file_out)

Notre base de données est maintenant stockée dans une bucket diffusion sur le ssp cloud, ce qui nous permet de ne pas avoir à repartir des fichiers compressés initiaux. 

**Pour importer directement les données sans passer par les étapes précédentes, on pourra utiliser le code suivant :**

In [5]:
# Importation bucket S3
MY_BUCKET = "mvallat/diffusion"
FILE_PATH_S3 = f"{MY_BUCKET}/projet/data_brevets.parquet"

with fs.open(FILE_PATH_S3, 'rb') as file_in:
    data_brevets = pd.read_parquet(file_in)

data_brevets.head(2)

,family_id,doc-number,kind,country,date-produced,status,invention-title,applicant_1_orgname,applicant_1_address-1,applicant_1_city,...,publication_doc-number,publication_date,publication_bopinum,publication_nature,application_country,application_doc-number,application_date,last-fee-payement,next-fee-payement,date-search-completed
0,NA,NA,NA,NA,20161223,NA,NA,NA,NA,NA,...,None,None,None,None,None,None,None,None,None,None
1,NA,NA,NA,NA,20161230,NA,NA,NA,NA,NA,...,None,None,None,None,None,None,None,None,None,None
